In [1]:
from Bio.Seq import Seq
import pandas as pd
from Bio.Blast import NCBIWWW, NCBIXML


with open("/home/marwene/Downloads/dna.example.fasta") as file:
    sequences = {}
    for line in file:
        line = line.strip("\n")
        if line.startswith(">"): # this is a name
            words = line.split()
            name = words[0][1:]
            sequences[name] = ""
        else: # this is a sequence
            sequences[name] = sequences[name] + line

# number of records in fasta file
print("In this file there are {} records".format(len(sequences.keys())))
lengths_sequences = []
for key, value in sequences.items():
    lengths_sequences.append(len(value))

# length of the sequences in a fasta file
print("The lengths of the sequences in this file are: {}".format(lengths_sequences))

# name and length of the longest sequence
for key, value in sequences.items():
    if len(value) == max(lengths_sequences):
        print("The name of the longest sequence in this file is: {} with a length of {}".format(key, len(value)))


# name and length of the shortest sequence
for key, value in sequences.items():
    if len(value) == min(lengths_sequences):
        print("The name of the shortest sequence in this file is: {} with a length of {}".format(key, len(value)))


# find the coordinates of all the ORF in a fasta file
ORF_coor = []
for name, sequence in sequences.items():
    for start_position in range(0, len(sequence) - 3):
        codon = sequence[start_position: start_position + 3]
        if codon == "ATG":
            for stop_position in range(start_position, len(sequence) - 3, 3):
                    codon = sequence[stop_position: stop_position + 3]
                    if codon in ["TAA", "TAG", "TGA"]:
                        ORF_coor.append((start_position, stop_position))
                        break

# Find all the ORF and append them to a list and then to a dictionnary with the key is the sequence 
#identifier and the value is the ORF sequence
ORF_dict = {}
ORF_seq = []


def GEN():
    for i in range(0, len(ORF_coor) - 1):
        yield i


a = GEN()

for name, sequence in sequences.items():
    for i in a:
        if ORF_coor[i][0] > ORF_coor[i + 1][0]:
            ORF_seq.append(sequence[ORF_coor[i][0]: ORF_coor[i][1]])
            ORF_dict[name] = ORF_seq
            ORF_seq = []
            break
        else:
            ORF_seq.append(sequence[ORF_coor[i][0]: ORF_coor[i][1]])

# This list contains the all the identifiers in a fasta file             
seq_name = []
# This list contains all the ORF translated using the Biopython 
ORF_translated = []
# This list contains all the ORFs sequences
ORF_gene = []

for name, ORF_list in ORF_dict.items():
    for ORF in ORF_list:
        seq_name.append(name)
        ORF_translated.append(Seq(ORF).translate())
        ORF_gene.append(ORF)

# create a data frame that contains name of the sequence, the sequence of the gene and the protein translated
ORF_data_frame = pd.DataFrame()
ORF_data_frame["Sequence name"] = seq_name
ORF_data_frame["Gene"] = ORF_gene
ORF_data_frame["Protein sequence"] = [''.join(aa) for aa in ORF_translated] 

ORF_data_frame.head()

In this file there are 25 records
The lengths of the sequences in this file are: [990, 724, 3080, 2863, 3832, 4805, 1663, 512, 691, 3072, 1801, 3603, 2478, 1608, 4745, 1810, 3424, 1451, 3276, 2124, 1712, 1325, 1189, 555, 2449]
The name of the longest sequence in this file is: gi|142022655|gb|EQ086233.1|323 with a length of 4805
The name of the shortest sequence in this file is: gi|142022655|gb|EQ086233.1|521 with a length of 512


,Sequence name,Gene,Protein sequence
0,gi|142022655|gb|EQ086233.1|43,ATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGC...,MASSAPIGSKPLKPSRIRRP
1,gi|142022655|gb|EQ086233.1|43,ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG,MRRSVIAALIE
2,gi|142022655|gb|EQ086233.1|43,ATGCCCAGCACGCCAATGCGTTCTTCATCCACA,MPSTPMRSSST
3,gi|142022655|gb|EQ086233.1|43,ATGCGTTCTTCATCCACA,MRSSST
4,gi|142022655|gb|EQ086233.1|43,ATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCC...,MTTKPSLASASPYTFPDVCSLQLPIGCALMMAGYFLPSSKFGGKWM...


In [2]:
# choose one example of e gene
gene = ORF_data_frame["Gene"][1]

In [3]:
result_handle = NCBIWWW.qblast("blastn", "nt", Seq(gene))
blast_record = NCBIXML.read(result_handle)

In [4]:
#  quick summary of all of the alignments greater than some threshold value.
E_VALUE_THRESH = 0.04

for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('****Alignment****')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query[0:75] + '...')
            print(hsp.match[0:75] + '...')
            print(hsp.sbjct[0:75] + '...')

****Alignment****
sequence: gi|1784293010|gb|CP046607.1| Burkholderia contaminans strain XL73 chromosome 1, complete sequence
length: 1541420
e value: 5.04371e-07
ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG...
|||||||||||||||||||||||||||||||||...
ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG...
****Alignment****
sequence: gi|1767888081|gb|CP028809.1| Burkholderia contaminans strain SK875 chromosome SK875-3, complete sequence
length: 1528467
e value: 5.04371e-07
ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG...
|||||||||||||||||||||||||||||||||...
ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG...
****Alignment****
sequence: gi|1712371596|gb|CP042166.1| Burkholderia contaminans strain ZCC chromosome 3, complete sequence
length: 1536075
e value: 5.04371e-07
ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG...
|||||||||||||||||||||||||||||||||...
ATGCGGCGCTCGGTCATCGCTGCATTGATCGAG...
****Alignment****
sequence: gi|1240697943|dbj|AP018359.1| Burkholderia contaminans CH-1 DNA, scaffold: scaffold03
length: 1538363
e value: 5.04371e-07
ATGCGGCGCTCGGTCATCGCT